In [1]:
pip install -r requirements.txt

You should consider upgrading via the '/Users/anirudhajoshi/Documents/Test/Assignment2-main/Case1/bin/python -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import os
import snowflake.connector
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
from snowflake.connector.pandas_tools import write_pandas

In [3]:
load_dotenv()

# Retrieve Snowflake credentials from environment variables
user = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = 'SF_WH_CASE1'
database = 'SF_DB_CASE1'
schema = 'SF_CASE1'
role = 'SYSADMIN'  

# Load CSV data into DataFrame
csv_file_path = '../Webscrape/CSV/extracted_updated.csv'  # Adjust this path as needed
df = pd.read_csv(csv_file_path)

# Prepare DataFrame for Snowflake by ensuring column names are uppercase
df.columns = [col.upper() for col in df.columns]

conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role=role,
)

conn.cursor().execute("CREATE DATABASE IF NOT EXISTS SF_DB_CASE1")
conn.cursor().execute("USE DATABASE SF_DB_CASE1")
conn.cursor().execute("""
    CREATE WAREHOUSE IF NOT EXISTS SF_WH_CASE1
    WITH WAREHOUSE_SIZE = 'MEDIUM'
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE
    WAREHOUSE_TYPE = 'STANDARD';
""")
conn.cursor().execute("CREATE SCHEMA IF NOT EXISTS SF_CASE1")
conn.cursor().execute("USE SCHEMA SF_CASE1")
conn.cursor().execute("""
    CREATE OR REPLACE TABLE SUMMARY_FINAL (
        NAME_OF_THE_TOPIC VARCHAR(500),
        YEAR VARCHAR(255),
        LEVEL VARCHAR(255),
        INTRODUCTION_SUMMARY VARCHAR(10000),
        LEARNING_OUTCOMES VARCHAR(10000),
        LINK_TO_THE_SUMMARY_PAGE VARCHAR(1000),
        LINK_TO_THE_PDF_FILE VARCHAR(1000)
    );
""")

print("Snowflake setup completed successfully.")

try:
    write_pandas(conn, df, table_name='SUMMARY_FINAL')
    print("Data transfer to Snowflake completed successfully.")
except Exception as e:
    print(f"Failed to transfer data to Snowflake: {e}")
finally:
    conn.close()

Snowflake setup completed successfully.


Data transfer to Snowflake completed successfully.
